<a href="https://colab.research.google.com/github/nlodder/ENPH353_Lab2/blob/main/ENPH353_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
from google.colab import userdata
import os

# Retrieve credentials
token = userdata.get('GH_TOKEN')
username = "nlodder"
repo = "ENPH353_Lab2"

# Clone the repo (using token for Write access)
!git clone https://{token}@github.com/{username}/{repo}.git

# Move into the directory
%cd {repo}

# Set Git Identity
!git config --global user.email "nathan.lodder@proton.me"
!git config --global user.name "Nathan Lodder"

Cloning into 'ENPH353_Lab2'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), 7.22 MiB | 35.73 MiB/s, done.
/content/ENPH353_Lab2/ENPH353_Lab2


In [88]:
import cv2 as cv
import numpy as np

dot_radius = 20
dot_rgb = (0, 0, 255)
rel_line_darkness = 20

def find_cent_x(hsv_slice):
  """
  @brief Scans a single row of a binary mask to find the line center.

  @param hsv_slice A 1D numpy array, slice near the bottom of the frame

  @return A int representing the horizontal center of the line, or None if no line is found.
  """
  # Extract the vallue (V) channel
  v_channel = hsv_slice[:, :, 2]
  avg_brightness = np.mean(v_channel)

  upper_v = int(max(0, avg_brightness - rel_line_darkness))
  upper_v = min(255, upper_v) # Ensure upper_v is within 0-255 for uint8

  # Ensure bounds are of the same type as hsv_slice (np.uint8)
  lower_bound = np.array([0, 0, 0], dtype=np.uint8)
  upper_bound = np.array([179, 255, upper_v], dtype=np.uint8)

  # binary mask: pixels in range white (255), others black (0)
  mask = cv.inRange(hsv_slice, lower_bound, upper_bound)
  fil1_write_obj.write(cv.cvtColor(mask, cv.COLOR_GRAY2BGR))

  # Find the center of the line
  moment = cv.moments(mask)

  # Check if the mask actually has white pixels to avoid DivisionByZero
  if moment["m00"] > 0:
    cent_horiz = int(moment["m10"] / moment["m00"]) # center
    return cent_horiz
  else:
    return None

# create video capture object and get fps and dimensions for output vid
cap_obj = cv.VideoCapture("/content/ENPH353_Lab2/raw_video_feed.mp4")
cap_fps = cap_obj.get(cv.CAP_PROP_FPS)
(cap_width, cap_height) = (int(cap_obj.get(cv.CAP_PROP_FRAME_WIDTH)), int(cap_obj.get(cv.CAP_PROP_FRAME_HEIGHT)))
cent_y = cap_height - dot_radius - 1

# create video writer object
write_obj  = cv.VideoWriter("labeled_video_feed.mp4", cv.VideoWriter_fourcc(*'mp4v'), cap_fps, (cap_width, cap_height))
fil1_write_obj = cv.VideoWriter("filter1_vid.mp4", cv.VideoWriter_fourcc(*'mp4v'), cap_fps, (cap_width, 2*dot_radius))

# get first frame
retval, raw_image = cap_obj.read()
cent_x = int(raw_image.shape[1]/2)

while retval==True:
  raw_slice = raw_image[cap_height-2*dot_radius-1 : cap_height ,:,:]
  hsv_slice = cv.cvtColor(raw_slice, cv.COLOR_BGR2HSV)
  test_cent_x = find_cent_x(hsv_slice)
  if test_cent_x != None:
    cent_x = test_cent_x
  cv.circle(raw_image, (cent_x, cent_y), dot_radius, dot_rgb, -1)
  write_obj.write(raw_image)
  retval, raw_image = cap_obj.read()

# Close video file once processing complete
cap_obj.release()
write_obj.release()
fil1_write_obj.release()

In [94]:
# Move your files into the cloned folder
# !cp labeled_video_feed.mp4 {repo}/
# !cp your_script.py {repo}/

# Navigate into the folder, stage, commit, and push
git add